# Different order
Trying to increase effectiveness and efficiency.

1. downscale and resave as JPG
2. rotate to landscape
3. orient upright
4. deskew
5. crop to first outer edges using Canny outlines
6. split into 8 major sections

In [1]:
from PIL import Image
from pathlib import Path
from os import path
import numpy as np
import cv2
import imutils
from pprint import pprint

## 1. downscale and resave as JPG

In [2]:
# Rename the image file
BATCH_DIR = Path("BatchProcess")
ORIGINAL = "original.jpeg"
img_path = Path(BATCH_DIR, ORIGINAL)
old = str(img_path)
old_name = str(img_path.name)
new_name = old_name.rstrip(str(img_path.suffix))

In [3]:
# load the image and convert to grayscale
new_img = Image.open(img_path).convert("L") # L is grayscale or "luminance"

/Users/wandalcooper/Library/Python/3.7/lib/python/site-packages/PIL/Image.py:2800: DecompressionBombWarning: Image size (141812970 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning,


In [4]:
# save it as a JPG
JPG_IMG = new_name+".jpg"
name = path.join(BATCH_DIR, JPG_IMG)
new_img.save(name)

In [5]:
# downscale
SCALE_PERCENT = 10 # percent of original size

# open the JPG, not JPEG
img_path = Path(BATCH_DIR, JPG_IMG)
img_name = str(img_path)
img = cv2.imread(img_name, cv2.IMREAD_UNCHANGED)

# calculate new size
width = int(img.shape[1] * SCALE_PERCENT / 100)
height = int(img.shape[0] * SCALE_PERCENT / 100)
new_size = (width, height)

# downscale image
DOWNSCALED = "downscaled.jpg"
downscaled = cv2.resize(img, new_size, interpolation = cv2.INTER_AREA)
file_name = str(Path(BATCH_DIR, DOWNSCALED))
cv2.imwrite(file_name, downscaled)

print(f"Original: {img.nbytes} bytes, downscaled: {downscaled.nbytes} bytes")

Original: 141812970 bytes, downscaled: 1417422 bytes


The images may be in any rotation of 90 degrees.  
This one happens to have the top at the left edge.
![](BatchProcess/downscaled.jpg)

## 2. rotate to landscape

In [6]:
# load the downscaled image
img_path = Path(BATCH_DIR, DOWNSCALED)
img = Image.open(str(img_path))
width = img.size[0]
height = img.size[1]

# rotate to landscape if needed, don't yet know rightsideup or upsidedown
if width < height:
    landscaped = img.rotate(90, expand=True)
LANDSCAPED = "landscaped.jpg"
file_name = str(Path(BATCH_DIR, LANDSCAPED))
landscaped.save(file_name)

A simple rotation based on the fact that the width needs to be greater than the height.  
This one happens to be upside down.
![](BatchProcess/landscaped.jpg)

## 3. orient upright

In [7]:
# load template image
# the template image was selected manually by using the GIMP editor
TEMPLATE = "template.jpg"
img_path = Path(BATCH_DIR, TEMPLATE)
template = str(img_path)
template_img = cv2.imread(template, 1) #1 is grayscale enum flag

The template is the image that is being searched for in another image.  
The template I am searching for is this "x" which is unique in a broad area of the image.
![](BatchProcess/template.jpg)

In [8]:
# load target image
img_path = Path(BATCH_DIR, LANDSCAPED)
target = str(img_path)
target_img = cv2.imread(target, 1) #1 is grayscale enum flag

# find these dimensions in GIMP
#     cropping box to reduce target search area, looking for template
left = 845
top = 790
right = left + 100
bottom = top + 100

# get the target image area from the template to reduce computational expense
#    crop_img = img[y:y+h, x:x+w] #opencv's x and y are flipped
target_search_area = target_img[top:bottom, left:right]

In [9]:
# calculate the likelihood of a match
method = cv2.TM_SQDIFF_NORMED  
result = cv2.matchTemplate(template_img, target_search_area, method) 

# minimum squared difference
#    image similarity score is maxVal, which is what I need
mn, maxVal, mnLoc, maxLoc = cv2.minMaxLoc(result)  

# exaggerate the values to make it easier to set a cutoff point
score = round((maxVal*100)**2)
print(f"Template match score: {score}")

# flip and save in place
# 600 points seems to be a good cutoff, arbitrarily chosen for now
if score < 600:
    RIGHT_SIDE_UP = "rightSideUp.jpg"
    rightsideup = cv2.rotate(target_img, cv2.ROTATE_180)
    file_name = str(Path(BATCH_DIR, RIGHT_SIDE_UP))
    cv2.imwrite(file_name, rightsideup)
else:
    # no need to rotate or save as its already in the correct position.
    pass

Template match score: 165


This image was upside down.  
The template matching process gave it a low score (below 600) so it needed to be rotated right side up.  
![](BatchProcess/rightSideUp.jpg)

## 4. deskew

In [10]:
# load image
img_path = Path(BATCH_DIR, RIGHT_SIDE_UP)
img = cv2.imread(str(img_path))

# convert to grayscale, flip foreground and background
#    foreground is now "white" and the background is "black"
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.bitwise_not(gray)

# threshold the image, setting all foreground pixels to 255 and all background pixels to 0
thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

# grab the (x, y) coordinates of all pixel values that are greater than zero, then use these coordinates to
#    compute a rotated bounding box that contains all coordinates
coords = np.column_stack(np.where(thresh > 0))
angle = cv2.minAreaRect(coords)[-1]

# the `cv2.minAreaRect` function returns values in the range [-90, 0); as the rectangle rotates clockwise the
#    returned angle trends to 0 -- in this special case we need to add 90 degrees to the angle
if angle < -45:
    angle = -(90 + angle)

# otherwise, just take the inverse of the angle to make it positive
else:
    angle = -angle
print(f"Angle skew: {angle}")

# rotate the image to deskew it
(h, w) = img.shape[:2]
center = (w // 2, h // 2)
M = cv2.getRotationMatrix2D(center, angle, 1.0)

DESKEWD = "deskewd.jpg"
deskewd = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
deskewd_name = str(Path(BATCH_DIR, DESKEWD))
cv2.imwrite(deskewd_name, deskewd)

CLEAN_DESKEWD = "cleanDeskewd.jpg"
clean_deskewd = deskewd.copy() # for use later, without the contour lines drawn in...
clean_deskewd_name = str(Path(BATCH_DIR, CLEAN_DESKEWD))
cv2.imwrite(clean_deskewd_name, deskewd)

Angle skew: 0.0


True

Fortunately, this image was not skewed... Need to try it on a badly skewed image.  
![](BatchProcess/deskewd.jpg)

## 5. crop to first outer edges using Canny outlines

#### 5a reduce noise while preserving boundaries

In [11]:
# load image
img_path = Path(BATCH_DIR, DESKEWD)
file_name = str(img_path)
uppercase_img = cv2.imread(file_name)

# convert to grayscale
grayscaled = cv2.cvtColor(uppercase_img, cv2.COLOR_BGR2GRAY)

# bilateralFilter reduces noise while preserving boundaries
bilateral_filtered = cv2.bilateralFilter(grayscaled, 11, 17, 17)
BILATERAL_FILTERED = "bilateralFiltered.jpg"
file_name = str(Path(BATCH_DIR, BILATERAL_FILTERED))
cv2.imwrite(file_name, bilateral_filtered)

True

It doesn't look like much has changed, but the "noise" was reduced.
![](BatchProcess/bilateralFiltered.jpg)

#### 5b invert the image

In [12]:
# invert the image for making bounding boxes
BILATERAL_FILTER_INVERTED = "bilateralFilteredInverted.jpg"
bilateral_filter_inverted = cv2.bitwise_not(gray)
file_name = str(Path(BATCH_DIR, BILATERAL_FILTER_INVERTED))
cv2.imwrite(file_name, bilateral_filter_inverted)

True

Inverting the black and white helps the Canny function find the edges.
With the image inverted, it is easier to see that the noise was reduced.  
If you zoom in on the original image, you would see where the black printer ink scatters around the characters a litte.
![](BatchProcess/bilateralFilteredInverted.jpg)

#### 5c find the edges with Canny

In [13]:
# get edges
CANNY = "canny.jpg"
canny_edges = cv2.Canny(bilateral_filter_inverted, 30, 200)
file_name = str(Path(BATCH_DIR, CANNY))
cv2.imwrite(file_name, canny_edges)

True

The Canny function produced white lines where if found the edges.  
If you look closely, you can see that the lines that make the characters and borders are thick enough that the Canny function produced two lines; one for each edge of every line of ink in the original image.
![](BatchProcess/canny.jpg)

#### 5d get the contours of the edges

In [14]:
# find contours in the edged image, keep only the largest ones, and initialize our screen contour
contours = cv2.findContours(canny_edges.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours) # convenience function (really simple, see https://github.com/jrosebr1/imutils/blob/master/imutils/convenience.py)
contours = sorted(contours, key = cv2.contourArea, reverse=True)[:10]
print(f"Contours found: {len(contours)}")
print(f"Length of first contour: {len(contours[0])}")
print(type(contours))

Contours found: 10
Length of first contour: 8
<class 'list'>


In [15]:
screenCnt = None
# loop over our contours
for c in contours:
    # approximate the contour
    epsilon = 0.015 * cv2.arcLength(c, True)
    approx = cv2.approxPolyDP(c, epsilon, True)
    
    # if our approximated contour has four points, then we can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break

In [16]:
# opencv colors, for convenience
BLUE = (255, 0, 0)
RED = (0, 0, 255)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
PURPLE = (255, 0, 255)

In [17]:
# Feed in the deskewd image
CONTOURED = "contoured.jpg"
contoured = cv2.drawContours(deskewd, contours, -1, GREEN, 1)
file_name = str(Path(BATCH_DIR, CONTOURED))
cv2.imwrite(file_name, contoured)

True

Although the current parameters for the Canny function didn't produce what I expected (intended), I think it is still of use as it found the edges of the boxes that I will need to crop out.
![](BatchProcess/contoured.jpg)

#### 5e get the edges of the intended cropping area

In [18]:
# convenience functions
def four_corners(box):
    xs = [el[0][0] for el in box]
    ys = [el[0][1] for el in box]
    x_min = min(xs)
    x_max = max(xs)
    y_min = min(ys)
    y_max = max(ys)
    return (x_min, x_max, y_min, y_max)

def area(corners):
    return (corners[1] - corners[0]) * (corners[3] - corners[2])

In [19]:
boxes = {}
for contour in contours:
    box = contours.index(contour)
    corners = four_corners(contour)
    boxes[box] = {"area": area(corners), "corners": corners}

pprint(boxes)

{0: {'area': 54918, 'corners': (577, 1063, 602, 715)},
 1: {'area': 56440, 'corners': (577, 1241, 326, 411)},
 2: {'area': 54805, 'corners': (578, 1063, 602, 715)},
 3: {'area': 76647, 'corners': (361, 1242, 75, 162)},
 4: {'area': 49306, 'corners': (625, 1179, 199, 288)},
 5: {'area': 55518, 'corners': (577, 1064, 450, 564)},
 6: {'area': 37825, 'corners': (90, 535, 326, 411)},
 7: {'area': 27048, 'corners': (579, 1062, 507, 563)},
 8: {'area': 48960, 'corners': (92, 668, 202, 287)},
 9: {'area': 41140, 'corners': (757, 1241, 326, 411)}}


/Users/wandalcooper/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
# the minmax across all the box corners
xmin = min([values["corners"][0] for box, values in boxes.items()])
xmax = max([values["corners"][1] for box, values in boxes.items()])
ymin = min([values["corners"][2] for box, values in boxes.items()])
ymax = max([values["corners"][3] for box, values in boxes.items()])

print(xmin, xmax, ymin, ymax)

90 1242 75 715


In [21]:
# Plot the four extremes of the first contour on the deskewd image, not the cropped image
#    This step is not needed, but I want to include it to emphasize/illustrate the point about finding the 4 corners using Canny's help
upper_left = (xmin, ymin)
lower_left = (xmin, ymax)
upper_right = (xmax, ymin)
lower_right = (xmax, ymax)

radius = 10
thickness = -1 # negative for filled circle

four_corners = cv2.circle(deskewd, upper_left, radius, RED, thickness)
four_corners = cv2.circle(deskewd, upper_right, radius, GREEN, thickness)
four_corners = cv2.circle(deskewd, lower_left, radius, BLUE, thickness)
four_corners = cv2.circle(deskewd, lower_right, radius, BLACK, thickness)
FOUR_CORNERS = "fourCorners.jpg"
file_name = str(Path(BATCH_DIR, FOUR_CORNERS))
cv2.imwrite(file_name, four_corners)

True

This is the image before the cropping.  The four dots show where the corners were calculated using the help of Canny.
![](BatchProcess/fourCorners.jpg)

In [22]:
# crop the image using the minmax of all the contours
left = xmin
top = ymin
right = xmax
bottom = ymax

# get the target image from the template
#     crop_img = img[y:y+h, x:x+w] #opencv's x and y are flipped
CROPPED = "cropped.jpg"
cropped = deskewd[top:bottom, left:right]
file_name = str(Path(BATCH_DIR, CROPPED))
cv2.imwrite(file_name, cropped)

True

This is the image after cropping using the contours found with the help of Canny.  
You can see that the edges have been cropped very close to the new boundaries and that the unwanted text at the bottom of the image was removed, too.
![](BatchProcess/cropped.jpg)

#### 5f crop the image

In [23]:
CLEANED_CROPPED = "cleanCropped.jpg"
cropped = clean_deskewd[top:bottom, left:right]
file_name = str(Path(BATCH_DIR, CLEANED_CROPPED))
cv2.imwrite(file_name, cropped)

True

Here is the cropped image without the contours drawn in.  
From this image, I can slice it up using coordinates found using GIMP.

![](BatchProcess/cleanCropped.jpg)

## 6. split into 8 major sections

The reason that I think that I need to crop the largest area of usable content first is that is was needed to make sure the images are aligned and sized properly for the following stages.  
From this point, I can just apply a template slicing to the sections to divide everything into 8 major pieces.  
Then, on each major piece, assuming the previous steps were successful in aligning everything properly, I can run step 5 (cropping using Canny) again on each of the smaller pieces to maintain the proper sizing and alignment on each smaller piece.  
Finally, I can use another template slicing specific to each of the 8 sections to extract the desired data.